# LiDAR point clouds with GRASS GIS & Python in Jupyter Notebooks

This tutorial is meant to show you how to check LiDAR point clouds with GRASS in python. When we get new airborne data we have to see what are we dealing with. Different sensors, planes, height of acqusiton, season, plant cover, weather and in general different situations are impacting LiDAR data. Before producing elevation models and before all the fun (simulatons and maps) we need to inspect the data.

Let's get started!

This tutorial can be run locally. You need to have GRASS GIS 8.4+ and Jupyter installed. The processes demands some specific libraries. Be sure to have numpy, pandas, geopandas, pdal, python-pdal and tiledbb with pybabylonjs. 

The first thing we need to do for any of the cases we'll see further on, is to import GRASS GIS python packages. In order to do so, we need to add GRASS GIS python package to PATH. Let's see how we do that.

## Setup

Install micromamba and setup a new invironment. It works aso with conda and mamba.

`"${SHELL}" <(curl -L micro.mamba.pm/install.sh)` <br>
`micromamba create -n grass_lidar numpy geopandas pdal python-pdal laspy tiledb-py pyarrow ipywidgets==7.7.2 jupyterlab==3.4.5 wxpython`

Import Python standard library and IPython packages we need.

In [ ]:
# pip install pybabylonjs
import os
import subprocess
import sys
import pdal
#pip install pdal python-pdal
import tiledb
#pip install tiledb-py pyarrow pandas pdal python-pdal
import laspy
#pip install lazrs
import pandas as pd
import geopandas as gpd
import numpy as np
import pybabylonjs

Check where GRASS GIS python packages are and add them to PATH.

In [ ]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

Import the GRASS GIS libraries we need.

In [ ]:
import grass.script as gs
import grass.jupyter as gj
from pathlib import Path

We can check the available commands with this command

In [ ]:
gs.get_commands()

## Project Initialization and Import

Create a temporary folder where to place our GRASS project.

In [ ]:
import tempfile
tempdir = tempfile.TemporaryDirectory()
# Create a project in the temporary directory
gs.create_project(path=tempdir.name, name="lidar_inspection", epsg="3794", overwrite=True)
# Start GRASS in the recently created project
session = gj.init(Path(tempdir.name,"lidar_inspection"))

Or create/connect a project in an existing GRASS database.

In [ ]:
gisdbase = '/your/path/grassdata'
gs.create_project(path=gisdbase, name="lidar_inspection", epsg="3794", overwrite=False)
session = gj.init(Path(gisdbase,"lidar_inspection"))

In [ ]:
# Let's check the environment settings: with os.environ[] we can see all of them
os.environ['GRASS_OVERWRITE'] # This environent variable shows us if we can overwrite our outputs. 

- Question: What can wee see from the output above? Is it the same as we are used in GRASS?

We can also use PyGRASS. For example, like this we get the documentation in Python.

In [ ]:
from grass.pygrass.modules import Module
r_in_pdal = Module("r.in.pdal")
print(r_in_pdal.__doc__) 

In [ ]:
# Set the location of our point cloud
point_cloud = "/your/path/point_cloud.laz"

In [ ]:
# Check our point cloud
print(gs.read_command("r.in.pdal", 
       input=point_cloud,
       flags="p"))

Know your data format!
https://www.asprs.org/wp-content/uploads/2019/07/LAS_1_4_r15.pdf

- Question 1: What is the difference between ReturnNumber and NumberOfReturns?
- Question 2: What is the classification flag value have points classified as ground?
- Question 3: What is the difference between the Point Format 7 and 8 and what can dimensions Red, Green, Blue, Infrared contain?

Another way to do it using PyGRASS.

In [ ]:
from grass.pygrass.modules import Module
Module("r.in.pdal", 
       input=point_cloud,
       flags="p")

In [ ]:
# See computational region
gs.run_command("g.region",flags="p")

In [ ]:
# Set the resolution
gs.run_command("g.region",res=1) 

We are interested to see how different filters, gridding (resolution) and methods affects our raszerization
Our point cloud has no CRS defined but it fits our project projection, se we overrired the projection chech with `-o`

Import our point cloud as a raster DSM, so we can check it a bit.

In [ ]:
%%time
# With -e we use the extent of the input for the raster extent 
# With -n we set the computation region to match the new raster map

gs.run_command("r.in.pdal",
               input=point_cloud,
               output='point_cloud_1m',
               resolution=1,
               flags="eno")

gs.run_command("r.in.pdal",
               input=point_cloud,
               output='point_cloud_1m_last',
               resolution=1,
               return_filter='last', # Let's see what points are last
               flags="eo")

gs.run_command("r.in.pdal",
               input=point_cloud,
               output='point_cloud_1m_ground',
               resolution=1,
               class_filter=2, # Let's see what points are classified as ground
               flags="eo")

In [ ]:
# Create a list of the rasters we produced
point_cloud_check=gs.list_strings(type=['raster'])

In [ ]:
series = gj.SeriesMap(height = 700)
series.add_rasters(point_cloud_check)
series.d_grid(size=250,color='orange') # With a grid will be easier to focus later on checking specific parts
series.show()
"series.save("/your/path/image.gif") # You can export that into a GIF, add it to your presentation and be cool

Set a smaller computational region for checking how would we like the data processed

In [ ]:
gs.run_command("g.region",save='my_region',flags="s") # Let's save our region in the GRASS way
region_dict = gs.parse_command("g.region",flags="gu") # Tthe parse_command parses the region parameters into a dictionary

In [ ]:
# Create a copy of the original region dictionary
smaller_region_dict = region_dict.copy()

# Update specific keys in the new dictionary
# Set the values of the extent you want to examine
updates = {
    'n': '79000',
    's': '78750',
    'w': '478250',
    'e': '478500'
}
# Apply the updates to the new dictionary
smaller_region_dict.update(updates)

Let's update momentarily our region so we can visualize a smaller portion of it

In [ ]:
n=int(smaller_region_dict['n'])
s=int(smaller_region_dict['s'])
w=int(smaller_region_dict['w'])
e=int(smaller_region_dict['e'])
univar_stat=gs.parse_command('r.univar',map='point_cloud_1m', flags='g')
z_min=float(univar_stat['min'])
z_max=float(univar_stat['max'])

gs.run_command("g.region",
               save='smaller_region',
               n=n,
               s=s,
               w=w,
               e=e,
               flags="d")
gs.run_command("g.region",region='smaller_region')

In [ ]:
gs.run_command("g.region",
               save='smaller_region',
               n=int(smaller_region_dict['n']),
               s=int(smaller_region_dict['s']),
               w=int(smaller_region_dict['w']),
               e=int(smaller_region_dict['e']),
               flags="d")
gs.run_command("g.region",region='smaller_region')

## 3D Data Visualization

Let's do a 3D check of the region: tiledb will manage that our computer survives

In [ ]:
data = os.path.expanduser("/your/path/point_cloud.laz")
array_name = os.path.expanduser("/your/path/tiledb_pc")
filename = "/your/path/point_cloud.laz""

Let's create a PDAL pipeline to import our LAZ into a tiledb database.

In [ ]:
%%time
pipeline = pdal.Reader.las(filename=data).pipeline()
pipeline |= pdal.Writer.tiledb(filename=array_name, x_tile_size=200, y_tile_size=200, z_tile_size=100)
pipeline.execute() # once we create it, we can just load it next time
# The output states the number of points

In [ ]:
from pybabylonjs import Show as show
A = tiledb.open(array_name)
df = A.query(attrs=('Red', 'Green','Blue')).df[w:e, s:n, z_min:z_max]
# df = A.query(attrs=('Infrared', 'Green','Red')).df[478000+500:478000+700, 77999+500:77999+700, 406.14:593.856000]
data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'], # or df['Infrared'], df['Green'], df['Red']
    'Green': df['Green'],
    'Blue': df['Blue']
}

show.point_cloud(source="dict",
                 data=data,
                 point_size = 4,
                 rgb_max=65535,
                 width = 1200,
                 height = 700)

- Question 1: What is the difference between a DSM and a DTM?
- Question 2: What resolution, method and filters would you use to generate the DTM and what to generate a DSM with vegetation classes?

In [ ]:
%%time

# Let's see what check how do different resolutions look
resolutions =  np.arange(0.50, 3.25, 0.25) # We set steps of 0.25m from 0.5 to 3m

# Create a list the will later be populated with outputs with different resolustions
rasters_different_res = []
    
# Iterate over the list of resolutions
for resolution in resolutions:
    # Format the name of the output
    resolution_str = f"{resolution:.2f}".replace('.', '')

    # Create the output name
    output_name = f"point_cloud_{resolution_str}"
    
    # Run the command with the current resolution and output name
    gs.run_command("r.in.pdal",
                   input=point_cloud,
                   output=output_name,
                   resolution=resolution,
                   class_filter=2,  # Filter points classified as ground
                   flags="eo")
    
    # Add the output name to the list of rasters
    rasters_different_res.append(output_name)

In [ ]:
series = gj.SeriesMap(height = 700)
series.add_rasters(rasters_different_res)
series.show()

What gridding would you chouse to create a DTM?

In [ ]:
%%time
# We use them all! Let patch them hierarchically and then interpolate.
gs.run_command("g.region",res=0.5)
gs.run_command("r.patch",input=rasters_different_res,output='dtm_patched',nprocs=8,mem=2000)
gs.run_command("r.relief",input='dtm_patched',output='dtm_patched_hs',altitude=30,azimuth=315)

In [ ]:
%%time
# List to store the names of the output shaded relief rasters
rasters_different_res_hs = []

# Iterate over each raster and generate the shaded relief
for raster in rasters_different_res:
    input_raster = raster
    output_name = f"{input_raster}_hs"

    # Run r.relief
    gs.run_command(
        "r.relief",
        input=input_raster,
        output=output_name,
        altitude=30,
        azimuth=315
    )

    # Add the output name to the list of rasters
    rasters_different_res_hs.append(output_name)

rasters_different_res_hs.append('dtm_patched_hs')

In [ ]:
series = gj.SeriesMap(height = 700)
series.add_rasters(rasters_different_res_hs)
series.show()
# You can see here that better resolution doesn't mean more detail.

Is that composite better or worse than an interpolated, raw one for your future data analysis?

In [ ]:
%%time
gs.run_command('r.resamp.bspline',
               input='point_cloud_100', # I decided that this one has a good resolution:detail payoff
               output='point_cloud_100_intp',
               method='bicubic')
gs.run_command("r.relief",input='point_cloud_100_intp',output='point_cloud_100_intp_hs',altitude=30,azimuth=315)

In [ ]:
series = gj.SeriesMap(height = 700)
series.add_rasters(['dtm_patched_hs','point_cloud_100_intp_hs'])
series.show()

In [ ]:
m = gj.InteractiveMap(width=700, height=700, tiles=None)
m.add_raster('dtm_patched_hs')
m.add_raster('point_cloud_100_intp_hs')
m.add_layer_control()
m.show()

Let's first interpolate the missing values in the patched DTM and then dig a bit more.

In [ ]:
%%time
gs.run_command('r.resamp.bspline',               
               input='dtm_patched',
               output='dtm_patched_int',
               method='bicubic')

Guess, what: GRASS has also a ton of addons and we can install them easily also here.

In [ ]:
# We can see differences on microtopography:
# Let's denoise and see how does local noise affect the elevation mode

# Let's install the extension
gs.run_command("g.extension",
               extension='r.denoise',
               operation='add')

Create a temporary directory or if you created one in the beginning just use that.

In [ ]:
# Let's compile the requirements of r.denoise
import tempfile
temp_dir = tempfile.mkdtemp() # If you created a new one in the beginning just use that
repo_url = "https://github.com/exuberant/mdenoise.git" # Life changes, repos change: check it
subprocess.run(["git", "clone", repo_url, temp_dir], check=True)

In [ ]:
# Change directory to the cloned repo
os.chdir(temp_dir)

# Compile the code (if needed)
compile_cmd = "g++ -o mdenoise mdenoise.cpp triangle.c"
subprocess.run(compile_cmd, shell=True, check=True)

print("Yep, no errors.")

Let's denoise and see how does local noise affect the elevation mode.

In [ ]:
%%time
thresholds =  np.arange(0.85, 1.0, 0.02)

# Create the list the will later be populated with outputs with different resolustions
dtm_different_denoise = []
    
# Iterate over the list of resolutions
for threshold in thresholds:
    # Format the name of the output
    threshold_str = f"{threshold:.2f}".replace('.', '')

    # Create the output name
    output_name = f"dtm_denoise_{threshold_str}"
    
    # Run the command with the current resolution and output name
    gs.run_command("r.denoise",
                   input='dtm_patched_int',
                   output=output_name,
                   iterations=5,
                   threshold=threshold)
    
    # Add the output name to the list of rasters
    dtm_different_denoise.append(output_name)

In [ ]:
%%time
# List to store the names of the output shaded relief rasters
dtm_different_denoise_hs = []

# Iterate over each raster and generate the shaded relief
for dtm in dtm_different_denoise:
    input_raster = dtm
    output_name = f"{input_raster}_hs"

    # Run r.relief
    gs.run_command(
        "r.relief",
        input=input_raster,
        output=output_name,
        altitude=30,
        azimuth=315
    )

    # Add the output name to the list of rasters
    dtm_different_denoise_hs.append(output_name)

In [ ]:
series = gj.SeriesMap(height = 700)
series.add_rasters(dtm_different_denoise_hs)
series.show()
# Pick the one that you need!